In [1]:
import pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive') 

directory = "/content/drive/MyDrive/cs598project/data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# FILTER PATIENTS UNDER 18
from tables.description import Time32Col
from dateutil.relativedelta import relativedelta
def filter_age(data):
    data['AGE'] = data.apply(
        lambda row: relativedelta(row['ADMITTIME'], row['DOB']).years, axis=1
        )

    data = data[data['AGE'] >= 18]
    return data

# FILTER ROWS WHERE PATIENT DIED INSIDE ICU 
def filter_death_in_icu(data): 
    survived_icu = data[
        ~( 
    (data['DEATHTIME'] >= data['INTIME']) &
    (data['DEATHTIME'] <= data['OUTTIME'])) 
    ]
    return survived_icu




In [4]:
procedures_icd = pandas.read_csv(f"{directory}/PROCEDURES_ICD.csv")
diagnoses_icd = pandas.read_csv(f"{directory}/DIAGNOSES_ICD.csv")
d_items = pandas.read_csv(f"{directory}/D_ITEMS.csv")

admissions = pandas.read_csv(f"{directory}/ADMISSIONS.csv")
icu_stays = pandas.read_csv(f"{directory}/ICUSTAYS.csv")
patients = pandas.read_csv(f"{directory}/PATIENTS.csv")

chartevents = pandas.read_csv(f"{directory}/chartevents_pruned.csv", low_memory=False)

print('PATIENTS:', patients.columns)
print('ADMISSIONS:', admissions.columns)
print('ICU_STAYS:', icu_stays.columns)



admissions['DISCHTIME'] = pandas.to_datetime(admissions['DISCHTIME'])
admissions['ADMITTIME'] = pandas.to_datetime(admissions['ADMITTIME'])
admissions['DEATHTIME'] = pandas.to_datetime(admissions['DEATHTIME'])

icu_stays['INTIME'] = pandas.to_datetime(icu_stays['INTIME'])
icu_stays['OUTTIME'] = pandas.to_datetime(icu_stays['OUTTIME'])
patients['DOB'] = pandas.to_datetime(patients['DOB'])
patients['DOD'] = pandas.to_datetime(patients['DOD'])

PATIENTS: Index(['ROW_ID', 'SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN',
       'EXPIRE_FLAG'],
      dtype='object')
ADMISSIONS: Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'],
      dtype='object')
ICU_STAYS: Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DBSOURCE',
       'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID',
       'INTIME', 'OUTTIME', 'LOS'],
      dtype='object')


In [5]:
patient_events = patients.merge(admissions, how='inner', on='SUBJECT_ID', suffixes=('', '_adm'))
patient_events = filter_age(patient_events)
patient_events = patient_events.merge(icu_stays, how='inner', on=['SUBJECT_ID', 'HADM_ID'], suffixes=('', '_icu'))
patient_events = filter_death_in_icu(patient_events)


In [6]:
patient_events = patient_events.merge(chartevents, how='inner', on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], suffixes=('', '_chart'))
patient_events = patient_events[patient_events['HAS_CHARTEVENTS_DATA'] == 1]
patient_events = patient_events[~patient_events['ITEMID'].isnull()]


In [7]:
# CHARTEVENT GROUPING

patient_events = patient_events[[
    'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 
    'INSURANCE', 'GENDER', 'ETHNICITY', 'AGE',
    'ITEMID', 'CHARTTIME','VALUE', 'VALUENUM'
]]

# Pivot chartevents rows by chart time 
pivot = patient_events.pivot_table(index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME'], columns='ITEMID',
                     values='VALUENUM').reset_index()
# print(res.head)

# filter ICU stays by demographic information
icu_stays = patient_events[[
    'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 
    'INSURANCE', 'GENDER', 'ETHNICITY', 'AGE'
]].drop_duplicates()


# Join ICU data with pivoted chartevents, each chart time / ICU combo will have a column for each ITEMID
dataset = icu_stays.merge(pivot, how='inner', on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], suffixes=('', ''))

print(dataset.head)

<bound method NDFrame.head of          SUBJECT_ID  HADM_ID  ICUSTAY_ID INSURANCE GENDER ETHNICITY  AGE  \
0               249   116935      215044  Medicare      F     WHITE   74   
1               249   116935      215044  Medicare      F     WHITE   74   
2               249   116935      215044  Medicare      F     WHITE   74   
3               249   116935      215044  Medicare      F     WHITE   74   
4               249   116935      215044  Medicare      F     WHITE   74   
...             ...      ...         ...       ...    ...       ...  ...   
2470455       44128   141304      253669   Private      M     WHITE   50   
2470456       44128   141304      253669   Private      M     WHITE   50   
2470457       44128   141304      253669   Private      M     WHITE   50   
2470458       44128   141304      253669   Private      M     WHITE   50   
2470459       44128   141304      253669   Private      M     WHITE   50   

                   CHARTTIME     51  184  ...  225309  22

In [8]:
dataset # Many NaNs present in various columns. Some more column corresponding to heart rate / blood pressure are much more prevalent

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INSURANCE,GENDER,ETHNICITY,AGE,CHARTTIME,51,184,...,225309,225310,226707,226755,226756,226757,226758,227013,227242,227243
0,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 12:00:00,133.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 13:00:00,141.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 14:00:00,163.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 15:00:00,153.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 16:00:00,158.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470455,44128,141304,253669,Private,M,WHITE,50,2149-06-12 11:00:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2470456,44128,141304,253669,Private,M,WHITE,50,2149-06-12 18:34:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2470457,44128,141304,253669,Private,M,WHITE,50,2149-06-12 20:55:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2470458,44128,141304,253669,Private,M,WHITE,50,2149-06-12 22:03:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:

dataset.columns

Index(['SUBJECT_ID',    'HADM_ID', 'ICUSTAY_ID',  'INSURANCE',     'GENDER',
        'ETHNICITY',        'AGE',  'CHARTTIME',           51,          184,
                211,          442,          454,          455,          618,
                723,         1352,         1495,         1524,         1880,
               6701,         6754,         8368,         8440,         8441,
               8555,       220045,       220050,       220051,       220179,
             220180,       220181,       220210,       220274,       220603,
             220734,       220739,       223761,       223762,       223830,
             223900,       223901,       224167,       224643,       225309,
             225310,       226707,       226755,       226756,       226757,
             226758,       227013,       227242,       227243],
      dtype='object')

In [10]:
# ICD9 GROUPING

diagnoses_icd = pandas.read_csv(f"{directory}/DIAGNOSES_ICD.csv")
diagnoses_icd = diagnoses_icd[~diagnoses_icd['ICD9_CODE'].isna()]

diag_cats = [
    range(1, 139),
    range(140, 239),
    range(240, 279),
    range(280, 289),
    range(290, 319),
    range(320, 389),
    range(390, 459),
    range(460, 519),
    range(520, 579),
    range(580, 629),
    range(630, 679),
    range(680, 709),
    range(710, 739),
    range(740, 759),
    range(760, 779),
    range(780, 799),
    range(800, 999),
]


In [11]:
# Group ICD-9 codes into broad categories according to https://en.wikipedia.org/wiki/List_of_ICD-9_codes
# 17 Groups were used in the paper, we find ourselves with 17 (numerical ranges) + 2 V/E external injury + 1 misc

def group_diag_icd9(code): 
    code  = str(code)
    if code[0] == 'V': 
        return 17
    if code[0] == 'E': 
        return 18

    for i, rng in enumerate(diag_cats): 
        if int(code[:3]) in rng: 
            return i
    return 19

diagnoses_icd['icd_group'] = diagnoses_icd['ICD9_CODE'].apply(lambda code: group_diag_icd9(code))

In [12]:
diagnoses_icd = diagnoses_icd[['SUBJECT_ID', 'HADM_ID', 'icd_group']]


In [13]:
# Merge one-hot ICD9 and join to dataset
res = pandas.get_dummies(diagnoses_icd.set_index(['HADM_ID'])['icd_group'], prefix='icd').max(level=0)

dataset = dataset.merge(res, on=['HADM_ID'], how='left')

dataset

<ipython-input-13-264cfa1e2940>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  res = pandas.get_dummies(diagnoses_icd.set_index(['HADM_ID'])['icd_group'], prefix='icd').max(level=0)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INSURANCE,GENDER,ETHNICITY,AGE,CHARTTIME,51,184,...,icd_10,icd_11,icd_12,icd_13,icd_14,icd_15,icd_16,icd_17,icd_18,icd_19
0,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 12:00:00,133.0,3.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 13:00:00,141.0,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 14:00:00,163.0,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 15:00:00,153.0,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 16:00:00,158.0,4.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470455,44128,141304,253669,Private,M,WHITE,50,2149-06-12 11:00:00,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2470456,44128,141304,253669,Private,M,WHITE,50,2149-06-12 18:34:00,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2470457,44128,141304,253669,Private,M,WHITE,50,2149-06-12 20:55:00,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2470458,44128,141304,253669,Private,M,WHITE,50,2149-06-12 22:03:00,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


In [14]:
labels = pandas.read_csv(f"{directory}/icu_labels/label_vector.csv", low_memory=False)
labels.shape
data_labeled = dataset.merge(labels,  on='ICUSTAY_ID', how='inner')

In [15]:
data_labeled

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INSURANCE,GENDER,ETHNICITY,AGE,CHARTTIME,51,184,...,icd_11,icd_12,icd_13,icd_14,icd_15,icd_16,icd_17,icd_18,icd_19,target
0,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 12:00:00,133.0,3.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
1,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 13:00:00,141.0,NaN,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
2,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 14:00:00,163.0,NaN,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
3,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 15:00:00,153.0,NaN,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
4,249,116935,215044,Medicare,F,WHITE,74,2149-12-22 16:00:00,158.0,4.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470455,44128,141304,253669,Private,M,WHITE,50,2149-06-12 11:00:00,NaN,NaN,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0
2470456,44128,141304,253669,Private,M,WHITE,50,2149-06-12 18:34:00,NaN,NaN,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0
2470457,44128,141304,253669,Private,M,WHITE,50,2149-06-12 20:55:00,NaN,NaN,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0
2470458,44128,141304,253669,Private,M,WHITE,50,2149-06-12 22:03:00,NaN,NaN,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0


In [16]:
data_labeled.to_csv(f"{directory}/processed/train_final.csv", index=False)